In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
G

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

## Count numbere of rows

In [4]:
df.count()

2557288

Transform to fit schema

In [7]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RED72VWWCOS7S|   32787517|B008HDQYLQ|     348668413| 2015-08-31|
| RZHWQ208LTEPV|   16374060|B005OBZBD6|     264704759| 2015-08-31|
|R37LBC3XAVLYOO|    9984817|B00RQL8U2G|      95173602| 2015-08-31|
|R3L7XJMA0MVJWC|   12635190|B0081SBO4Y|     835659279| 2015-08-31|
|R2I2GHSI7T1UBN|   43905102|B008E6OK3U|     539243347| 2015-08-31|
|R2GFFKHK4I6VMX|   52596997|B00W6NTULY|     337446474| 2015-08-31|
|R1R0UDX2XAN1S4|   43871104|B00GXUMYKA|     468857193| 2015-08-31|
|R22C8FMBSTFRY8|   11346008|B005EIX8JS|     125753094| 2015-08-31|
|R118NNIQ75XPGO|   49206471|B000HJBKMQ|     834273114| 2015-08-31|
|R30HYXHZQ49621|   37596267|B004LY59V6|     612086079| 2015-08-31|
|R3EMLKY0GF1E90|   31554283|B00CAVM85M|     280334010| 2015-08-31|
|R23BX7EGJMGQJR|   43211735|B00DP6X1LG|     233116679| 2015-08

In [8]:
product = df.select(["product_id", "product_title"])
product.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B008HDQYLQ|Garden Weasel Gar...|
|B005OBZBD6|10 Foot Mc4 Solar...|
|B00RQL8U2G|GE String A Long ...|
|B0081SBO4Y|Key Pair Lawn Wit...|
|B008E6OK3U|Zodiac R0502300 L...|
|B00W6NTULY|Hirts Gardens Swe...|
|B00GXUMYKA|AGPtEK 12 PCS Smo...|
|B005EIX8JS|Design Toscano Ea...|
|B000HJBKMQ|TERRO T300 Liquid...|
|B004LY59V6|BLACK+DECKER LBXR...|
|B00CAVM85M|Reach 'n Spray Pe...|
|B00DP6X1LG|Puro-Kleen Ultra-...|
|B00025H2PY|Diatomaceous Eart...|
|B0079GHJXY|Perky-Pet 312C Pa...|
|B004HFJ762|Crossbow Dow Spec...|
|B00004RAGL|Apex REM 15 15-Fo...|
|B010PWBNNK|Elucto Electric B...|
|B00T77AWY6|Ohuhu® 100 Ft Exp...|
|B000UJH6HQ|Toro 53746 Drip B...|
|B000HJBKMQ|TERRO T300 Liquid...|
+----------+--------------------+
only showing top 20 rows



In [11]:
customers = df.groupby("customer_id").agg({"customer_id":"count"})
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46263557|             1|
|   44418648|             1|
|   34220092|             1|
|   10770444|             1|
|   12819130|             1|
|   24692214|             3|
|   51981844|             3|
|   43622169|             1|
|   43097449|             1|
|   45978717|             1|
|   20227781|             1|
|   11487525|             4|
|   31169607|             1|
|   22552080|             2|
|   15829398|             2|
|   46909180|             1|
|     198573|             3|
|   33383395|             1|
|    2934525|             3|
|    2333626|             1|
+-----------+--------------+
only showing top 20 rows



In [12]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RED72VWWCOS7S|          1|            2|          8|   N|
| RZHWQ208LTEPV|          5|            0|          0|   N|
|R37LBC3XAVLYOO|          5|            4|          5|   N|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|
|R22C8FMBSTFRY8|          5|            2|          2|   N|
|R118NNIQ75XPGO|          3|            0|          0|   N|
|R30HYXHZQ49621|          2|            0|          0|   N|
|R3EMLKY0GF1E90|          5|            0|          0|   N|
|R23BX7EGJMGQJR|          5|            1|          2|   N|
|R2Z4B6SDEAZF6E|          5|            0|          0|   N|
|R35289PGJERP5J|          5|            

In [13]:
mode="append"
lg_url = "database-1.cv1pxdhixoh6.us-west-2.rds.amazonaws.com"
config = {"user":"postgres12345",
          "password": "H4edpDSzMkiWNKuQ5OrH",
          "driver":"org.postgresql.Driver"}

In [15]:
vine_table.write.jdbc(url=lg_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored